# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")


city_data_df["Max Temp"]=city_data_df["Max Temp"]-272.15

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,khovd,48.00556,91.64194,-1.59,19,20,0.63,MN,1710567764
1,1,kruisfontein,-34.00333,24.73142,19.82,64,59,10.09,ZA,1710567765
2,2,severo-yeniseyskiy,60.37552,93.03017,-9.21,89,100,5.81,RU,1710567765
3,3,palikir - national government center,6.92477,158.16109,29.47,84,100,5.14,FM,1710567766
4,4,bilibino,68.05464,166.43721,-13.14,86,70,1.42,RU,1710567767


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    color="City"
)

# Display the map
city_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_df=city_data_df[(city_data_df["Max Temp"]>=18)&(city_data_df["Max Temp"]<=24)&(city_data_df["Cloudiness"]>=20)& \
(city_data_df["Cloudiness"]<=80)&(city_data_df["Humidity"]>=45)&(city_data_df["Humidity"]<=85)&(city_data_df["Wind Speed"]<=10)]

# Drop any rows with null values
filtered_city_df.dropna()

# Display sample data
filtered_city_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,cidade velha,14.91531,-23.60527,22.21,78,20,4.12,CV,1710567768
7,7,hermanus,-34.41870,19.23446,18.11,83,75,4.59,ZA,1710567769
11,11,edinburgh of the seven seas,-37.06757,-12.31155,18.51,85,46,6.78,SH,1710567771
45,45,albany,-35.02692,117.88369,23.19,81,49,1.79,AU,1710567792
52,52,mossel bay,-34.18307,22.14605,18.96,74,48,6.40,ZA,1710567796
61,61,port alfred,-33.58601,26.88329,21.71,85,63,3.58,ZA,1710567801
63,63,howrah,-42.88728,147.40554,20.04,63,75,7.20,AU,1710567802
74,74,bredasdorp,-34.53215,20.04031,19.52,81,71,3.40,ZA,1710567809
216,216,kingston,-29.05459,167.96628,23.23,64,75,6.69,NF,1710567895
256,256,hamilton,32.29490,-64.78303,18.11,85,20,6.69,BM,1710567920


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=filtered_city_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
6,cidade velha,CV,14.91531,-23.60527,78,
7,hermanus,ZA,-34.41870,19.23446,83,
11,edinburgh of the seven seas,SH,-37.06757,-12.31155,85,
45,albany,AU,-35.02692,117.88369,81,
52,mossel bay,ZA,-34.18307,22.14605,74,
61,port alfred,ZA,-33.58601,26.88329,85,
63,howrah,AU,-42.88728,147.40554,63,
74,bredasdorp,ZA,-34.53215,20.04031,81,
216,kingston,NF,-29.05459,167.96628,64,
256,hamilton,BM,32.29490,-64.78303,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "categories":"accommodation.hotel",
    "limit":20,
    "filter":"",
    "bias":"",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city_lat=row["Lat"]
    city_lng=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{city_lng},{city_lat},{radius}"
    params["bias"] = f"proximity:{city_lng},{city_lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cidade velha - nearest hotel: Baía do Coral
hermanus - nearest hotel: Aloe guest house
edinburgh of the seven seas - nearest hotel: No hotel found
albany - nearest hotel: Sixdegrees
mossel bay - nearest hotel: Protea Hotel Mossel Bay
port alfred - nearest hotel: No hotel found
howrah - nearest hotel: Shoreline Hotel
bredasdorp - nearest hotel: Victoria Hotel
kingston - nearest hotel: Governor's Lodge Resort Hotel
hamilton - nearest hotel: The Oxford House
ulladulla - nearest hotel: Hotel Marlin
laiwu - nearest hotel: 格林豪泰酒店
la'ie - nearest hotel: No hotel found
macgregor - nearest hotel: Belconnen Soccer Club
santa adelia - nearest hotel: No hotel found
san pedro pochutla - nearest hotel: Pochutla
sinan - nearest hotel: No hotel found
sao vicente - nearest hotel: Quinta Vale Vitis


,City,Country,Lat,Lng,Humidity,Hotel Name
6,cidade velha,CV,14.91531,-23.60527,78,Baía do Coral
7,hermanus,ZA,-34.41870,19.23446,83,Aloe guest house
11,edinburgh of the seven seas,SH,-37.06757,-12.31155,85,No hotel found
45,albany,AU,-35.02692,117.88369,81,Sixdegrees
52,mossel bay,ZA,-34.18307,22.14605,74,Protea Hotel Mossel Bay
61,port alfred,ZA,-33.58601,26.88329,85,No hotel found
63,howrah,AU,-42.88728,147.40554,63,Shoreline Hotel
74,bredasdorp,ZA,-34.53215,20.04031,81,Victoria Hotel
216,kingston,NF,-29.05459,167.96628,64,Governor's Lodge Resort Hotel
256,hamilton,BM,32.29490,-64.78303,85,The Oxford House


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width=700,
    frame_height=500,
    color="City",
    size="Humidity",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)